# 论文内容复现
notebook只算解读代码部分，更多算法流程请看技术报告。  
首先导入训练集和测试集，考虑到电脑性能问题，每个类取了100个训练集，共1000个，测试集也取了1000个。

In [1]:
import numpy as np
import cv2
import random
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original',data_home='./datasets')
imgs,labels = mnist['data'], mnist['target']


train_imgs = imgs[:100]
train_labels = labels[:100]
text_imgs = imgs[60000:61000]
text_labels = labels[60000:61000]
#从每个类中取100个训练集出来
for i in range(9):
    temp_x = imgs[(i+1)*6000:(i+1)*6000+100]
    temp_y = labels[(i+1)*6000:(i+1)*6000+100]
    train_imgs = np.append(train_imgs,temp_x)
    train_labels = np.append(train_labels,temp_y)
    
#将训练集和测试集都reshape成1000*28*28的矩阵，方便下面提取 patch 
train_imgs = train_imgs.reshape(1000,28,28)
text_imgs = text_imgs.reshape(1000,28,28)
print(train_imgs.shape)
print(text_imgs.shape)

(1000, 28, 28)
(1000, 28, 28)


## 特征学习
特征学习的第一个步骤，从训练图像中随机提取 patch。这里去了以7为边长的正方形 patch。

In [2]:
patch_first = np.zeros((40000,49))
patch_second = np.zeros((64000,49))
patch_text = np.zeros((64000,49))

#训练集随机取patch
def patch_random(data,num,count):
    for j in range(num):
        point = random.randint(0,20)
        patch_temp = data[point:point+7,point:point+7]
        patch_temp = patch_temp/255                  #归一化
        patch_temp = patch_temp.reshape(49,)
        patch_first[count*40+j] = patch_temp

#训练集按步长取patch
def patch_step(data,num,count,stride):
    for i in range(num//8):
        for j in range(num//8):
            patch_temp = data[i*stride:i*stride+7,j*stride:j*stride+7]
            patch_temp = patch_temp/255            #归一化
            patch_temp = patch_temp.reshape(1,49)
            patch_second[count*64+j] = patch_temp
               
#测试集取步长取patch
def patch_step2(data,num,count,stride):
    for i in range(num//8):
        for j in range(num//8):
            patch_temp = data[i*stride:i*stride+7,j*stride:j*stride+7]
            patch_temp = patch_temp/255 #归一化
            patch_temp = patch_temp.reshape(1,49)
            patch_text[count*64+j] = patch_temp

  训练集随机取 patch，1000张图片，每张图片40个 patch，每个 patch 49个像素，再使用 KMeans 聚类算法得到十个聚类中心

In [4]:

for i in range(1000):
    patch_random(train_imgs[i],40,i)

print(patch_first.shape)    
clf = KMeans(n_clusters = 10, random_state = 0)  
clf.fit(patch_first)
cluster_center = clf.cluster_centers_
print(cluster_center.shape)   # 10个聚类中心，一个 patch 49个像素，得到（10,49）的矩阵


(40000, 49)
(10, 49)


## 特征提取与分类
对训练集进行以3为步长，提取共64个7*7大小的patch

In [5]:
for i in range(1000):
    patch_step(train_imgs[i],64,i,3)
    
print(patch_second.shape)

(64000, 49)


对提取出的 patch 计算其与10个聚类中心中哪个距离最近，则归为该类

In [6]:
def center(point, cluster_center):
    min_distance = np.sum((point-cluster_center[0])**2)
    for i in range(1,cluster_center.shape[0]):
        distance_temp = np.sum((point-cluster_center[i])**2)
        if(distance_temp < min_distance):
            min_distance = distance_temp
            index = i
            return i

imgs_patch = np.zeros((1000,64,10)) #1000个训练集，每个64个patch，10个通道
for i in range(1000):
    imgs_each = np.zeros((64,10))  #64个 patch 对应10个类的矩阵，若 patch 属于某类，则该类相应的矩阵位置为1，其他类的值为0
    for j in range(64):
        point = center(patch_second[i*64+j], cluster_center)
        imgs_each[j,point] = 1
    imgs_patch[i]=imgs_each
    
print(imgs_patch.shape)

(1000, 64, 10)


对 patch 进行 pool 来减少特征的数目，将64个 patch 分为四个象限，每个象限的patch对应的值相加，最终一幅图的特征从$64*10$维降到了$4*10$维

In [7]:
#池化
def pooling(imgs_every):
    patch_final1 = imgs_every[0]+imgs_every[1]+imgs_every[2]+imgs_every[3]
    patch_final2 = imgs_every[4]+imgs_every[5]+imgs_every[6]+imgs_every[7]
    patch_final3 = imgs_every[32]+imgs_every[33]+imgs_every[34]+imgs_every[35]
    patch_final4 = imgs_every[36]+imgs_every[37]+imgs_every[38]+imgs_every[39]
    for i in range(1,4):
        patch_final1 = patch_final1+imgs_every[8*i]+imgs_every[8*i+1]+imgs_every[8*i+2]+imgs_every[8*i+3]
        patch_final2 = patch_final2+imgs_every[8*i+4]+imgs_every[8*i+5]+imgs_every[8*i+6]+imgs_every[8*i+7]
        patch_final3 = patch_final3+imgs_every[8*i+32]+imgs_every[8*i+33]+imgs_every[8*i+34]+imgs_every[8*i+35]
        patch_final4 = patch_final4+imgs_every[8*i+36]+imgs_every[8*i+37]+imgs_every[8*i+38]+imgs_every[8*i+39]
    
    return np.concatenate((patch_final1,patch_final2,patch_final3,patch_final4))

train_final = np.zeros((1000,40))
for i in range(1000):
    imgs_every = imgs_patch[i]
    train_final[i] = pooling(imgs_every)

将训练集导入到svm中学习

In [8]:
clf = svm.SVC()
clf.fit(train_final,train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## 对测试集进行特征提取

In [9]:
# 对测试集取patch
for i in range(1000):
    patch_step2(text_imgs[i],64,i,3)
    
print(patch_text.shape)

(64000, 49)


In [10]:
imgs_patch_text = np.zeros((1000,64,10)) #1000个训练集，每个64个patch，10个通道
for i in range(1000):
    imgs_each2 = np.zeros((64,10))
    for j in range(64):
        point_2 = center(patch_text[i*64+j], cluster_center)
        imgs_each2[j,point_2] = 1
    imgs_patch_text[i]=imgs_each2

对测试集进行池化

In [11]:
#测试集池化
text_final = np.zeros((1000,40)) # 最终输入SVM的测试集
for i in range(1000):
    imgs_every_text = imgs_patch_text[i]
    text_final[i] = pooling(imgs_every_text)

In [12]:
prediction=clf.predict(text_final)
accurancy = np.sum(np.equal(prediction, text_labels)) / 1000
print(accurancy)

0.431


最终测试结果精确度并不高，只有0.431，分析原因有很多影响最终的精确度。首先，提取的 patch 大小可以调整，稍大一点的 patch 重叠部分更多，会提高精确度。其次，对训练集按步长提取时，可以将步长设的更小一点，那么每张图取出的 patch 数量就越多，特征就越多，可以更好地训练模型。此外，训练集的数量应该也会有一部分原因影响，训练集数量一开始我只取了1000个，取得更多效果应该会更好。聚类的中心数对精确度应该也有影响。以及池化的过程，我这里是直接将4个象限的 patch 加起来实现了，可以在池化的过程进一步改进。  

因为我工程能力比较差，所以写代码比较慢，然后在写代码的过程中无意中将很多函数的矩阵给确定了下来，本来是想可以进一步调参提升准确度，但因为这周做了 ppt 分享，然后本身将算法转化为代码的能力还有所欠缺，deadline 近在眼前，只能将调参的思路写在上面。